In [2]:
import requests
import json
import pandas as pd
import numpy as np
import yfinance as yf
import pandas as pd
import os

C:\Users\mc590\anaconda3\lib\site-packages\pandas\core\computation\expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


## Construct Dispersions Trading on SP5
Data Applied:

* SPX EOD underlying prices, vanilla option chains with greeks, source: https://www.optionsdx.com/shop/
* SP500 or all the single stock option historical data with greeks, source:
https://optiondata.org/
* SP500 index components and weighting, source: locally saved

Other (maybe) useful data source:
* dolthub database, on options: https://www.dolthub.com/repositories/post-no-preference/options/query/master?active=Tables&q=SELECT+*+FROM+%60option_chain%60+WHERE+act_symbol+%3D+%27EBAY%27+and+date+%3D+%272019-02-09%27
* yahoo finance API
* MarketData API: https://docs.marketdata.app/api/

In [11]:
data_spx = pd.read_excel("index_comp.xlsx")
data_spx.head(10)

,#,Company,Symbol,Weight,Price,Chg,% Chg
0,1,Apple Inc,AAPL,7.565046,181.11,1.02,-0.0056
1,2,Microsoft Corp,MSFT,6.992379,335.12,2.54,-0.0076
2,3,Amazon.com Inc,AMZN,3.091382,124.19,1.42,-0.0116
3,4,Nvidia Corp,NVDA,2.763275,392.30,-5.40,(-1.36%)
4,5,Alphabet Inc Cl A,GOOGL,2.081271,124.62,0.90,-0.0073
5,6,Alphabet Inc Cl C,GOOG,1.823885,125.19,0.82,-0.0066
6,7,Meta Platforms Inc Class A,META,1.714472,272.40,-0.21,(-0.08%)
7,8,Berkshire Hathaway Inc Cl B,BRK.B,1.644951,329.25,6.13,-0.019
8,9,Tesla Inc,TSLA,1.577013,213.80,6.28,-0.0302
9,10,Unitedhealth Group Inc,UNH,1.303234,499.83,6.20,-0.0125


In [68]:
class Dispersions_Booking:
    
    #initialize the booking with cash and start date
    def __init__(self, initial_cash, start_date, index_component):
        self.cash = initial_cash
        self.option_position = {}
        self.equity_position = {}
        self.today = start_date
        self.index_component = index_component
        self.portfolio_value_and_greeks = {"value":initial_cash,"delta":0,"gamma":0,"theta":0,"vega":0}
        
    #build the component leg
    def component_leg_construction(self, data_path, date, quantity):
        #load data for the date
        try:
            equity = pd.read_csv(data_path + "\\Single Stock\\" + date + "stocks.cvs")
            equity.index = equity["symbol"]
            options = pd.read_csv(data_path + "\\Single Stock\\" + date + "options.cvs")
        
            tickers = list(self.index_component["Symbol"])
            weights = [w/100 for w in list(self.index_component["Weight"])]

            for i in range(len(tickers)):
                ticker = tickers[i]
                weight = weights[i]

                try:
                    #choose the stock ticker
                    data = options[options["underlying"] == ticker]

                    #choose the last expiry
                    target_expiry = data["expiration"].iloc[-1]
                    data = data[data["expiration"] == target_expiry]

                    #choose cloest OTM put and call to construct a straddle/strangle
                    call_options = data[data["type"] == "call"]
                    put_options = data[data["type"] == "put"]
                    spot = equity.loc[ticker, "close"]
                    call_choice = call_options[call_options["strike"] >= spot].iloc[0]
                    put_choice = put_options[put_options["strike"] <= spot].iloc[-1]
                    call_ticker = call_choice["contract"]
                    put_ticker = put_choice["contract"]
                except Exception as e:
                    #print(ticker, e)
                    continue

                #purchase the call and put and delta hedge, notice that we do the purchase with ask
                #the amount purchased, the quantity: weight
                if quantity >= 0: #buy at ask
                    cash_update = -1 * weight * quantity * (call_choice["ask"] + put_choice["ask"])
                else: #sell at bid
                    cash_update = -1 * weight * quantity * (call_choice["bid"] + put_choice["bid"])
                delta_to_hedge = weight * quantity * (call_choice["delta"] + put_choice["delta"]) #the delta amount to hedge

                #purchase the spot to delta hedge
                cash_update += spot * delta_to_hedge

                #update the positions
                self.cash += cash_update
                if ticker not in self.equity_position.keys():
                    self.equity_position[ticker] = -1 * delta_to_hedge
                else:
                    self.equity_position[ticker] -= delta_to_hedge
                if call_ticker not in self.option_position.keys():
                    self.option_position[call_ticker] = weight * quantity
                else:
                    self.option_position[call_ticker] += weight * quantity
                if put_ticker not in self.option_position.keys():
                    self.option_position[put_ticker] = weight * quantity
                else:
                    self.option_position[put_ticker] += weight * quantity
        except:
            print("No data loaded for date: ", date)
            
    #build the index leg
    def index_leg_construction(self, data_path, date, quantity):
        #load data for the date
        month_selected = pd.to_datetime(date).strftime("%Y%m")
        SP5_data = pd.read_csv(data_path + "spx_eod_" + month_selected + ".txt")
        
        #select the data for the date
        data = SP5_data[SP5_data["[QUOTE_DATE]"] == date]

        #choose the last expiry
        target_expiry = data["[EXPIRE_DATE]"].iloc[-1]
        data = data[data["[EXPIRE_DATE]"] == target_expiry]
        spot = data["[UNDERLYING_LAST]"].iloc[-1]
        spot_ticker = "SPX"

        #choose cloest OTM put and call to construct a straddle/strangle
        call_choice = data[data["[STRIKE]"] >= spot].iloc[0]
        put_choice = data[data["[STRIKE]"] <= spot].iloc[-1]
        call_ticker = spot_ticker + str(call_choice["[EXPIRE_DATE]"]) + "C" + str(call_choice["[STRIKE]"])
        put_ticker = spot_ticker + str(put_choice["[EXPIRE_DATE]"]) + "P" + str(put_choice["[STRIKE]"])

        #purchase the call and put and delta hedge, notice that we do the purchase with ask
        #the amount purchased, the quantity: weight
        if quantity >= 0: #buy at ask
            cash_update = -1 * quantity * (call_choice["[C_ASK]"] + put_choice["[P_ASK]"])
        else: #sell at bid
            cash_update = -1 * quantity * (call_choice["[C_BID]"] + put_choice["[P_BID]"])
        delta_to_hedge = quantity * (call_choice["[C_DELTA]"] + put_choice["[P_DELTA]"]) #the delta amount to hedge

        #purchase the spot to delta hedge
        cash_update += spot * delta_to_hedge
    
        #update the positions
        self.cash += cash_update
        if spot_ticker not in self.equity_position.keys():
            self.equity_position[spot_ticker] = -1 * delta_to_hedge
        else:
            self.equity_position[spot_ticker] -= delta_to_hedge
        if call_ticker not in self.option_position.keys():
            self.option_position[call_ticker] = quantity
        else:
            self.option_position[call_ticker] += quantity
        if put_ticker not in self.option_position.keys():
            self.option_position[put_ticker] = quantity
        else:
            self.option_position[put_ticker] += quantity
    
    
    #daily delta hedge
    def delta_hedge(self, data_path, date):
        try:
            equity = pd.read_csv(data_path + "\\Single Stock\\" + date + "stocks.cvs")
            equity.index = equity["symbol"]
            options = pd.read_csv(data_path + "\\Single Stock\\" + date + "options.cvs")
            month_selected = pd.to_datetime(date).strftime("%Y%m")
            SP5_data = pd.read_csv(data_path + "spx_eod_" + month_selected + ".txt")
            new_equity_positions = {}

            for option_ticker in self.option_position.keys(): #could be call or put
                option_fetch = options[options["contract"] == option_ticker]
                if len(option_fetch) == 0:
                    self.option_position.pop(option_ticker, None)
                    print("Option expired: ", option_ticker)
                    continue
                else:
                    quantity = self.option_position[option_ticker]
                    underlying = str(option_fetch.iloc[0, 1]) #grab the underlying
                    target_spot_quantity_from_this_contract = -1 * float(option_fetch["delta"].iloc[0]) * quantity
                    if underlying not in new_equity_positions:
                        new_equity_positions[underlying] = target_spot_quantity_from_this_contract
                    else:
                        new_equity_positions[underlying] += target_spot_quantity_from_this_contract
                        
            for equity_ticker in self.equity_position.keys():
                if equity_ticker in new_equity_positions.keys():
                    target_spot_quantity = new_equity_positions[equity_ticker]
                else: #the option is expired, and you need to clear the equity hedging position
                    target_spot_quantity = 0
                    
                current_spot_quantity = self.equity_position[equity_ticker]
                spot_quantity_change = target_spot_quantity - current_spot_quantity
                spot = equity.loc[equity_ticker, "close"]
                #delta hedge now:
                self.cash -= spot * spot_quantity_change
                
                #update equity position
                if target_spot_quantity != 0:
                    self.equity_position[equity_ticker] = target_spot_quantity
                else: #remove the position
                    del equity_position[equity_ticker]
                    
        except:
            print("No data loaded for date: ", date)    
    
            
    #calculate the portfolio value for specific date, along with the greeks
    def portfolio_value_calculator(self, data_path, date):
        #load data for the date
        try:
            equity = pd.read_csv(data_path + "\\Single Stock\\" + date + "stocks.cvs")
            equity.index = equity["symbol"]
            options = pd.read_csv(data_path + "\\Single Stock\\" + date + "options.cvs")
        
            #initialization
            cash_on_book = self.cash
            self.portfolio_value_and_greeks["value"] = cash_on_book
            self.portfolio_value_and_greeks["delta"] = 0
            self.portfolio_value_and_greeks["gamma"] = 0
            self.portfolio_value_and_greeks["theta"] = 0
            self.portfolio_value_and_greeks["vega"] = 0

            #calculate the option position value
            for option_ticker in self.option_position.keys():
                option_fetch = options[options["contract"] == option_ticker]
                if len(option_fetch) == 0: #option has expired or not fetched
                    #remove the option if not fetched
                    self.option_position.pop(option_ticker, None)
                    continue
                else:
                    quantity = self.option_position[option_ticker]
                    if quantity >= 0:
                        value = float(option_fetch["bid"].iloc[0])
                    else:
                        value = float(option_fetch["ask"].iloc[0])
                    delta = float(option_fetch["delta"].iloc[0])
                    gamma = float(option_fetch["gamma"].iloc[0])
                    theta = float(option_fetch["theta"].iloc[0])
                    vega = float(option_fetch["vega"].iloc[0])
                    self.portfolio_value_and_greeks["value"] += value * quantity
                    self.portfolio_value_and_greeks["delta"] += delta * quantity
                    self.portfolio_value_and_greeks["gamma"] += gamma * quantity
                    self.portfolio_value_and_greeks["theta"] += theta * quantity
                    self.portfolio_value_and_greeks["vega"] += vega * quantity

            #calculate the equity position value
            for equity_ticker in self.equity_position.keys():
                equity_fetch = equity[equity["symbol"] == equity_ticker]
                quantity = self.equity_position[equity_ticker]
                if len(equity_fetch) == 0:
                    print("Equity Not Found ", equity_fetch)
                    self.equity_position.pop(equity_ticker, None)
                    continue
                else:
                    value = float(equity_fetch["close"].iloc[0])
                    delta = 1
                    self.portfolio_value_and_greeks["value"] += value * quantity
                    self.portfolio_value_and_greeks["delta"] += delta * quantity 
        except:
            print("No data loaded for date: ", date)
    
    #build the dispersion, vega flat
    
    
    #build the dispersion, theta flat
    
    
    #calculate implied correlation
        
        
    #Visualize the PNL
    
    

In [69]:
date = "2013-01-02"
data_path = r"D:\Dispersions_Data"
Dispersion = Dispersions_Booking(10000, date, data_spx)

In [70]:
Dispersion.component_leg_construction(data_path, date, 10)

In [71]:
Dispersion.portfolio_value_calculator(data_path, date)

In [72]:
Dispersion.portfolio_value_and_greeks

{'value': 9978.66499162102,
 'delta': -2.0541565410453488e-16,
 'gamma': 0.47475761402000055,
 'theta': -93.37612758875999,
 'vega': 1201.7428715649316}

In [73]:
date = "2013-01-03"

In [74]:
Dispersion.delta_hedge(data_path, date)
Dispersion.portfolio_value_calculator(data_path, date)

In [75]:
Dispersion.portfolio_value_and_greeks

{'value': 9978.149812493575,
 'delta': -1.6674758362283937e-15,
 'gamma': 0.49069205396000026,
 'theta': -89.3657693177199,
 'vega': 1212.1734289096391}

In [76]:
business_dates = pd.bdate_range(start = "2013-01-02", end = "2013-06-30")
business_dates = [date.strftime("%Y-%m-%d") for date in business_dates]

In [77]:
tracker = []

for date in business_dates[:5]:
    Dispersion.delta_hedge(data_path, date)
    Dispersion.portfolio_value_calculator(data_path, date)
    report = (Dispersion.portfolio_value_and_greeks).copy()
    tracker.append(report)

In [78]:
pd.DataFrame(tracker)

,value,delta,gamma,theta,vega
0,9978.803464,-1.511107e-16,0.474758,-93.376128,1201.742872
1,9978.288285,-1.667476e-15,0.490692,-89.365769,1212.173429
2,9976.175176,1.011832e-16,0.486877,-93.106202,1203.844583
3,9975.590957,-4.138942e-17,0.483948,-93.005934,1199.474785
4,9974.050995,1.145324e-15,0.483548,-92.373336,1195.905595


In [87]:
Dispersion.option_position

{'AAPL150117C00550000': 0.7565046,
 'AAPL150117P00540000': 0.7565046,
 'MSFT150117C00028000': 0.6992379,
 'MSFT150117P00025000': 0.6992379,
 'AMZN150117C00260000': 0.3091382,
 'AMZN150117P00250000': 0.3091382,
 'NVDA150117C00015000': 0.2763275,
 'NVDA150117P00012000': 0.2763275,
 'GOOG150117C00730000': 0.2081271,
 'GOOG150117P00720000': 0.2081271,
 'BRKB150117C00095000': 0.1644951,
 'BRKB150117P00092500': 0.1644951,
 'TSLA150117C00037000': 0.1577013,
 'TSLA150117P00035000': 0.1577013,
 'UNH150117C00055000': 0.1303234,
 'UNH150117P00050000': 0.1303234,
 'XOM150117C00090000': 0.120267,
 'XOM150117P00087500': 0.120267,
 'JNJ150117C00072500': 0.114164,
 'JNJ150117P00070000': 0.114164,
 'JPM150117C00045000': 0.11402540000000001,
 'JPM150117P00042000': 0.11402540000000001,
 'V150117C00160000': 0.10399640000000002,
 'V150117P00155000': 0.10399640000000002,
 'LLY150117C00050000': 0.0972688,
 'LLY150117P00047000': 0.0972688,
 'PG150117C00070000': 0.09596189999999999,
 'PG150117P00067500': 0.095

In [ ]:
def index_leg_construction(data_path, date, quantity):
    #load data for the date
    month_selected = pd.to_datetime(date).strftime("%Y%m")
    SP5_data = pd.read_csv(data_path + "spx_eod_" + month_selected + ".txt")

In [97]:
#load data for the date
data_path = r"D:\Dispersions_Data\SPX\\"
month_selected = pd.to_datetime(date).strftime("%Y%m")
SP5_data = pd.read_csv(data_path + "spx_eod_" + month_selected + ".txt", sep=r',', skipinitialspace=True)

In [98]:
SP5_data

,[QUOTE_UNIXTIME],[QUOTE_READTIME],[QUOTE_DATE],[QUOTE_TIME_HOURS],[UNDERLYING_LAST],[EXPIRE_DATE],[EXPIRE_UNIX],[DTE],[C_DELTA],[C_GAMMA],...,[P_LAST],[P_DELTA],[P_GAMMA],[P_VEGA],[P_THETA],[P_RHO],[P_IV],[P_VOLUME],[STRIKE_DISTANCE],[STRIKE_DISTANCE_PCT]
0,1357160400,2013-01-02 16:00,2013-01-02,16.0,1462.33,2013-01-04,1357333200,2.0,1.00000,0.00000,...,0.0,-0.00109,0.00004,0.00338,-0.02391,0.00000,0.99703,NaN,362.3,0.248
1,1357160400,2013-01-02 16:00,2013-01-02,16.0,1462.33,2013-01-04,1357333200,2.0,1.00000,0.00000,...,0.0,-0.00103,0.00000,0.00365,-0.02423,-0.00008,0.92371,NaN,337.3,0.231
2,1357160400,2013-01-02 16:00,2013-01-02,16.0,1462.33,2013-01-04,1357333200,2.0,1.00000,0.00000,...,0.0,-0.00059,0.00006,0.00344,-0.02367,0.00000,0.85118,NaN,312.3,0.214
3,1357160400,2013-01-02 16:00,2013-01-02,16.0,1462.33,2013-01-04,1357333200,2.0,1.00000,0.00000,...,0.0,-0.00128,0.00000,0.00360,-0.02358,0.00000,0.77927,NaN,287.3,0.196
4,1357160400,2013-01-02 16:00,2013-01-02,16.0,1462.33,2013-01-04,1357333200,2.0,1.00000,0.00000,...,0.0,-0.00131,0.00005,0.00397,-0.02357,0.00000,0.76524,NaN,282.3,0.193
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29702,1359666000,2013-01-31 16:00,2013-01-31,16.0,1498.33,2015-12-18,1450472400,1051.0,0.07338,0.00032,...,0.0,-0.94159,0.00000,0.00000,-0.07962,-60.46820,0.00025,NaN,601.7,0.402
29703,1359666000,2013-01-31 16:00,2013-01-31,16.0,1498.33,2015-12-18,1450472400,1051.0,0.04753,0.00029,...,0.0,-0.94181,0.00000,0.00000,-0.07933,-63.34806,0.00024,NaN,701.7,0.468
29704,1359666000,2013-01-31 16:00,2013-01-31,16.0,1498.33,2015-12-18,1450472400,1051.0,0.03897,0.00024,...,0.0,-0.94149,0.00000,0.00000,-0.07989,-64.78736,0.00050,NaN,751.7,0.502
29705,1359666000,2013-01-31 16:00,2013-01-31,16.0,1498.33,2015-12-18,1450472400,1051.0,0.01737,0.00007,...,0.0,-0.94201,0.00000,0.00000,-0.07960,-71.98657,-0.00028,NaN,1001.7,0.669


In [99]:
SP5_data.columns

Index(['[QUOTE_UNIXTIME]', '[QUOTE_READTIME]', '[QUOTE_DATE]',
       '[QUOTE_TIME_HOURS]', '[UNDERLYING_LAST]', '[EXPIRE_DATE]',
       '[EXPIRE_UNIX]', '[DTE]', '[C_DELTA]', '[C_GAMMA]', '[C_VEGA]',
       '[C_THETA]', '[C_RHO]', '[C_IV]', '[C_VOLUME]', '[C_LAST]', '[C_SIZE]',
       '[C_BID]', '[C_ASK]', '[STRIKE]', '[P_BID]', '[P_ASK]', '[P_SIZE]',
       '[P_LAST]', '[P_DELTA]', '[P_GAMMA]', '[P_VEGA]', '[P_THETA]',
       '[P_RHO]', '[P_IV]', '[P_VOLUME]', '[STRIKE_DISTANCE]',
       '[STRIKE_DISTANCE_PCT]'],
      dtype='object')

In [100]:
SP5_data['[EXPIRE_DATE]'].iloc[0]

'2013-01-04'

In [103]:
data = SP5_data[SP5_data['[QUOTE_DATE]'] == "2013-01-04"]

In [104]:
data

,[QUOTE_UNIXTIME],[QUOTE_READTIME],[QUOTE_DATE],[QUOTE_TIME_HOURS],[UNDERLYING_LAST],[EXPIRE_DATE],[EXPIRE_UNIX],[DTE],[C_DELTA],[C_GAMMA],...,[P_LAST],[P_DELTA],[P_GAMMA],[P_VEGA],[P_THETA],[P_RHO],[P_IV],[P_VOLUME],[STRIKE_DISTANCE],[STRIKE_DISTANCE_PCT]
2772,1357333200,2013-01-04 16:00,2013-01-04,16.0,1466.1,2013-01-04,1357333200,0.0,1.00000,0.00000,...,0.0,-0.00037,0.00001,0.00185,-0.02565,-0.00020,1.74085,NaN,366.1,0.250
2773,1357333200,2013-01-04 16:00,2013-01-04,16.0,1466.1,2013-01-04,1357333200,0.0,1.00000,0.00000,...,0.0,-0.00039,0.00005,0.00207,-0.02470,-0.00002,1.61319,NaN,341.1,0.233
2774,1357333200,2013-01-04 16:00,2013-01-04,16.0,1466.1,2013-01-04,1357333200,0.0,1.00000,0.00000,...,0.0,-0.00126,0.00003,0.00259,-0.02564,0.00000,1.48940,NaN,316.1,0.216
2775,1357333200,2013-01-04 16:00,2013-01-04,16.0,1466.1,2013-01-04,1357333200,0.0,1.00000,0.00000,...,0.0,-0.00112,0.00005,0.00242,-0.02519,-0.00046,1.36441,NaN,291.1,0.199
2776,1357333200,2013-01-04 16:00,2013-01-04,16.0,1466.1,2013-01-04,1357333200,0.0,1.00000,0.00000,...,0.0,-0.00067,0.00001,0.00187,-0.02453,-0.00004,1.34038,NaN,286.1,0.195
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4233,1357333200,2013-01-04 16:00,2013-01-04,16.0,1466.1,2015-12-18,1450472400,1078.0,0.07417,0.00039,...,0.0,-0.93907,0.00000,0.00000,-0.07886,-62.02044,-0.00031,NaN,633.9,0.432
4234,1357333200,2013-01-04 16:00,2013-01-04,16.0,1466.1,2015-12-18,1450472400,1078.0,0.04801,0.00027,...,0.0,-0.93932,0.00000,0.00000,-0.07963,-64.97359,0.00032,NaN,733.9,0.501
4235,1357333200,2013-01-04 16:00,2013-01-04,16.0,1466.1,2015-12-18,1450472400,1078.0,0.03881,0.00026,...,0.0,-0.93897,0.00000,0.00000,-0.07883,-66.44991,0.00008,NaN,783.9,0.535
4236,1357333200,2013-01-04 16:00,2013-01-04,16.0,1466.1,2015-12-18,1450472400,1078.0,0.02021,0.00013,...,0.0,-0.93961,0.00000,0.00000,-0.07882,-73.83372,-0.00048,NaN,1033.9,0.705


In [111]:
try:
    #select the data for the date
    data = SP5_data[SP5_data["[QUOTE_DATE]"] == "2013-01-04"]
    
    #choose the last expiry
    target_expiry = data["[EXPIRE_DATE]"].iloc[-1]
    data = data[data["[EXPIRE_DATE]"] == target_expiry]
    spot = data["[UNDERLYING_LAST]"].iloc[-1]
    spot_ticker = "SPX"

    #choose cloest OTM put and call to construct a straddle/strangle
    call_choice = data[data["[STRIKE]"] >= spot].iloc[0]
    put_choice = data[data["[STRIKE]"] <= spot].iloc[-1]
    call_ticker = spot_ticker + str(call_choice["[EXPIRE_DATE]"]) + "C" + str(call_choice["[STRIKE]"])
    put_ticker = spot_ticker + str(put_choice["[EXPIRE_DATE]"]) + "P" + str(put_choice["[STRIKE]"])
    
    #purchase the call and put and delta hedge, notice that we do the purchase with ask
    #the amount purchased, the quantity: weight
    if quantity >= 0: #buy at ask
        cash_update = -1 * quantity * (call_choice["[C_ASK]"] + put_choice["[P_ASK]"])
    else: #sell at bid
        cash_update = -1 * quantity * (call_choice["[C_BID]"] + put_choice["[P_BID]"])
    delta_to_hedge = quantity * (call_choice["[C_DELTA]"] + put_choice["[P_DELTA]"]) #the delta amount to hedge

    #purchase the spot to delta hedge
    cash_update += spot * delta_to_hedge

    #update the positions
    self.cash += cash_update
    if spot_ticker not in self.equity_position.keys():
        self.equity_position[spot_ticker] = -1 * delta_to_hedge
    else:
        self.equity_position[spot_ticker] -= delta_to_hedge
    if call_ticker not in self.option_position.keys():
        self.option_position[call_ticker] = quantity
    else:
        self.option_position[call_ticker] += quantity
    if put_ticker not in self.option_position.keys():
        self.option_position[put_ticker] = quantity
    else:
        self.option_position[put_ticker] += quantity
except:
    print("No data loaded for date: ", date)

No data loaded for date:  2013-01-08


In [114]:
quantity = 1
cash = 0
equity_position = {}
option_position = {}

#select the data for the date
data = SP5_data[SP5_data["[QUOTE_DATE]"] == "2013-01-04"]

#choose the last expiry
target_expiry = data["[EXPIRE_DATE]"].iloc[-1]
data = data[data["[EXPIRE_DATE]"] == target_expiry]
spot = data["[UNDERLYING_LAST]"].iloc[-1]
spot_ticker = "SPX"

#choose cloest OTM put and call to construct a straddle/strangle
call_choice = data[data["[STRIKE]"] >= spot].iloc[0]
put_choice = data[data["[STRIKE]"] <= spot].iloc[-1]
call_ticker = spot_ticker + str(call_choice["[EXPIRE_DATE]"]) + "C" + str(call_choice["[STRIKE]"])
put_ticker = spot_ticker + str(put_choice["[EXPIRE_DATE]"]) + "P" + str(put_choice["[STRIKE]"])

#purchase the call and put and delta hedge, notice that we do the purchase with ask
#the amount purchased, the quantity: weight
if quantity >= 0: #buy at ask
    cash_update = -1 * quantity * (call_choice["[C_ASK]"] + put_choice["[P_ASK]"])
else: #sell at bid
    cash_update = -1 * quantity * (call_choice["[C_BID]"] + put_choice["[P_BID]"])
delta_to_hedge = quantity * (call_choice["[C_DELTA]"] + put_choice["[P_DELTA]"]) #the delta amount to hedge

#purchase the spot to delta hedge
cash_update += spot * delta_to_hedge

#update the positions
cash += cash_update
if spot_ticker not in equity_position.keys():
    equity_position[spot_ticker] = -1 * delta_to_hedge
else:
    equity_position[spot_ticker] -= delta_to_hedge
if call_ticker not in option_position.keys():
    option_position[call_ticker] = quantity
else:
    option_position[call_ticker] += quantity
if put_ticker not in option_position.keys():
    option_position[put_ticker] = quantity
else:
    option_position[put_ticker] += quantity

In [115]:
equity_position

{'SPX': -0.03209000000000001}

In [116]:
option_position

{'SPX2015-12-18C1475.0': 1, 'SPX2015-12-18P1450.0': 1}